<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/Sarima_MRL_LINE_BudgetandForecasting_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Load the data from the provided dataset
data_file_path = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_with_outliers_replaced.xlsx'
forecast_file_path = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\forecast.csv'

# Step 1: Load the data
df_data = pd.read_excel(data_file_path)
df_forecast = pd.read_csv(forecast_file_path)

# Step 2: Preprocess the data
# Drop rows with missing values in the specified columns
df_data = df_data.dropna(subset=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])

# Step 3: Create MRL groups based on 'MRL_LINE'
mrl_groups = df_data.groupby('MRL_LINE')

# Step 4: Create a DataFrame to store the final results
forecast_data = []

# Step 5: Perform SARIMA forecasting for each MRL group and month
for name, group in mrl_groups:
    # Get the historical data for the MRL group
    historical_data = group.drop(['COUNTRY', 'VISION_OUC', 'CURRENCY'], axis=1)

    # Filter forecast data for the current MRL group and category
    mrl_category = tuple(historical_data.iloc[0, -4:].values)
    forecast_group = df_forecast[df_forecast['Category'] == str(mrl_category)]

    # Loop through forecast data and add the SARIMA forecast for each month
    for _, row in forecast_group.iterrows():
        year = row['Year']
        month = row['Month']
        sarima_forecast = row['SARIMA_Forecast']

        # Check if historical_data is not empty and contains data for the given year and month
        if not historical_data.empty and len(historical_data.loc[(historical_data['Year'] == year) & (historical_data[str(month).zfill(2)] != 0)]) > 0:
            # Get the historical actual expenditure for the given year and month
            actual_expenditure = historical_data.loc[(historical_data['Year'] == year) & (historical_data[str(month).zfill(2)] != 0), str(month).zfill(2)].values[0]
        else:
            actual_expenditure = None

        # Append the forecast result and actual expenditure for the current MRL group, year, and month to the final data
        forecast_data.append([name, year, month, sarima_forecast, actual_expenditure])

# Step 6: Consolidate the results and save them to the final output sheet
df_final_forecast = pd.DataFrame(forecast_data, columns=['MRL_LINE', 'Year', 'Month', 'SARIMA_Forecast', 'Actual_Expenditure'])

# Save the final output to a new CSV file
df_final_forecast.to_csv('C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Sarima\\Actual_vs_SARIMA_Forecast.csv', index=False)